In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from pandas_summary import DataFrameSummary
import pandas as pd
import numpy as np
import csv
pd.set_option('display.max_columns', 5000)

In [2]:
target_vars = ['DIED']
input_vars_reg_dx = '^DX\d'
input_vars_reg_dxccs = '^DXCCS\d'
input_vars_reg_pr = '^PR\d'
input_vars_reg_prccs = '^PRCCS\d'

In [21]:
path = '../../../data/hcup/nis/NIS2014/total.csv'

In [22]:
#path = '../../../data/hcup/nis/all_year_combination/fn_pt_missingdied.pickle'

In [23]:
demo_cols = ["AGE", "DIED", "FEMALE", "LOS", "ZIPINC_QRTL", "PAY1", "TOTCHG", "RACE"]

In [24]:
# columns for diagnoses; for years before 2009, there are 15 columns, there are 25 for 2009-2013, 30 for 2014 and 2015

dx_cols = ['DX1',
 'DX2',
 'DX3',
 'DX4',
 'DX5',
 'DX6',
 'DX7',
 'DX8',
 'DX9',
 'DX10',
 'DX11',
 'DX12',
 'DX13',
 'DX14',
 'DX15',
 'DX16',
 'DX17',
 'DX18',
 'DX19',
 'DX20',
 'DX21',
 'DX22',
 'DX23',
 'DX24',
 'DX25',
 'DX26',
 'DX27',
 'DX28',
 'DX29',
 'DX30']

In [25]:
# columns for chronic condition identifier; for years before 2009, there are 15 columns, there are 25 for 2009-2013, 30 for 2014 and 2015

chron_cols = ['CHRON1',
 'CHRON2',
 'CHRON3',
 'CHRON4',
 'CHRON5',
 'CHRON6',
 'CHRON7',
 'CHRON8',
 'CHRON9',
 'CHRON10',
 'CHRON11',
 'CHRON12',
 'CHRON13',
 'CHRON14',
 'CHRON15',
 'CHRON16',
 'CHRON17',
 'CHRON18',
 'CHRON19',
 'CHRON20',
 'CHRON21',
 'CHRON22',
 'CHRON23',
 'CHRON24',
 'CHRON25',
 'CHRON26',
 'CHRON27',
 'CHRON28',
 'CHRON29',
 'CHRON30']

In [26]:
useful_cols = demo_cols + dx_cols + chron_cols

In [27]:
#fn_pt = pd.read_pickle(path)

In [28]:
fn_pt = pd.read_csv(path, dtype = str, quoting = csv.QUOTE_ALL, usecols = useful_cols)

In [29]:
# Myocardial infarction
icd9_mi = ['410', '412']
# Congestive heart failure
icd9_chf = ['428']
# Peripheral vascular disease
icd9_pvd = ['4439', '441', '7854', 'V434']
# Cerebrovascular disease
icd9_cd = ['430', '438']
# Dementia
icd9_dem = ['290']
# Chronic pulmonary disease
icd9_cpd = ['490', '491', '492', '493', '494', '495', '496', '497', '498', '499', '500', '501', '502', '503', 
            '504', '505', '5064']
# Rheumatic disease
icd9_rd = ['7100', '7101', '7104', '7140', '7141', '7142', '71481', '7125']
# Peptic ulcer disease
icd9_pud = ['531', '532', '533', '534']
# Mild liver disease
icd9_mld = ['5712', '5714', '5715', '5716']
# Diabetes without chronic complication
icd9_dw0cc = ['2500', '2501', '2502', '2503', '2507']
# Diabetes with chronic complication
icd9_dw1cc = ['2504', '2505', '2506']
# Hemiplegia or paraplegia
icd9_hop = ['3441', '342']
# Renal disease
icd9_rd = ['582', '5830', '5831', '5832', '5833', '5834', '5835', '5836', '5837', '585', '586', '588']
# Any malignancy
icd9_am = [str(x) for x in list(range(140,173))] + [str(x) for x in list(range(174,195))] + ['1958'] + [str(x) for x in list(range(200,209))]
# Moderate or severe liver disease
icd9_msld = ['4560', '4561', '45620', '45621', '5722', '5723', '5724', '5725', '5726', '5727', '5728']
# Metastatic solid tumor
icd9_mst = ['196', '197', '198', '1990', '1991']
# AIDS/HIV
icd9_hiv = ['042', '043', '044']

In [30]:
dx_reg = '^DX\d'
ci_reg = '^CHRON\d'
def add_cci(row):
#    print(row.row_id)
    if row.row_id%10000 == 0:
        print(row.row_id/7071763)
    point1 = icd9_mi + icd9_chf + icd9_pvd + icd9_cd + icd9_dem + icd9_cpd + icd9_rd + icd9_pud + icd9_mld + icd9_dw0cc
    point2 = icd9_dw1cc + icd9_hop + icd9_rd + icd9_am
    point3 = icd9_msld
    point6 = icd9_mst + icd9_hiv
    point = 0
    for icd9, cci in zip(row.filter(regex = dx_reg), row.filter(regex = ci_reg)):
        if pd.isnull(icd9) or cci == '0':
            continue
        else:
            for code in point1 + point2 + point3 + point6:
                if icd9.startswith(code): 
                    if code in point1:
                        point += 1
                    elif code in point2:
                        point += 2
                    elif code in point3:
                        point += 3
                    elif code in point6:
                        point += 6

    return point

In [31]:
fn_pt = fn_pt.astype(str)

In [32]:
fn_pt['row_id'] = list(fn_pt.index)

In [ ]:
fn_pt['CCI'] = fn_pt.apply(lambda row: add_cci(row),axis=1)

0.0
0.001414074538414254
0.002828149076828508
0.004242223615242762
0.005656298153657016
0.00707037269207127
0.008484447230485524
0.009898521768899778
0.011312596307314031
0.012726670845728287
0.01414074538414254
0.015554819922556794
0.016968894460971048
0.018382968999385303
0.019797043537799555
0.02121111807621381
0.022625192614628063
0.024039267153042318
0.025453341691456573
0.026867416229870825
0.02828149076828508
0.029695565306699333
0.031109639845113588
0.032523714383527844
0.033937788921942096
0.03535186346035635
0.036765937998770606
0.03818001253718486
0.03959408707559911
0.04100816161401336
0.04242223615242762
0.04383631069084187
0.045250385229256125
0.046664459767670384
0.048078534306084636
0.04949260884449889
0.05090668338291315
0.0523207579213274
0.05373483245974165
0.0551489069981559
0.05656298153657016
0.057977056074984414
0.059391130613398666
0.060805205151812924
0.062219279690227176
0.06363335422864143
0.06504742876705569
0.06646150330546993
0.06787557784388419
0.06928965

In [ ]:
len(fn_pt)

In [ ]:
path

In [ ]:
#fn_pt.to_csv(path, index = False, quoting = csv.QUOTE_ALL)

In [ ]:
fn_pt.to_pickle("../../../data/hcup/nis/NIS2014/total_filtered_withCCI.pickle")

#### Filter variables

In [1]:
path = "../../../data/hcup/nis/NIS_2015/total_withage_2015.csv"

In [10]:
fn_pt = pd.read_csv(path, dtype = str, quoting = csv.QUOTE_ALL, usecols = demo_cols)

In [85]:
path = "../../../data/hcup/nis/NIS_2015/total_withage_2015_withCCI.pickle"

In [86]:
data = pd.read_pickle(path)

In [87]:
data["RACE"] = fn_pt.RACE

In [88]:
data.columns

Index(['CHRON1', 'CHRON2', 'CHRON3', 'CHRON4', 'CHRON5', 'CHRON6', 'CHRON7',
       'CHRON8', 'CHRON9', 'CHRON10', 'CHRON11', 'CHRON12', 'CHRON13',
       'CHRON14', 'CHRON15', 'CHRON16', 'CHRON17', 'CHRON18', 'CHRON19',
       'CHRON20', 'CHRON21', 'CHRON22', 'CHRON23', 'CHRON24', 'CHRON25',
       'CHRON26', 'CHRON27', 'CHRON28', 'CHRON29', 'CHRON30', 'DX1', 'DX2',
       'DX3', 'DX4', 'DX5', 'DX6', 'DX7', 'DX8', 'DX9', 'DX10', 'DX11', 'DX12',
       'DX13', 'DX14', 'DX15', 'DX16', 'DX17', 'DX18', 'DX19', 'DX20', 'DX21',
       'DX22', 'DX23', 'DX24', 'DX25', 'DX26', 'DX27', 'DX28', 'DX29', 'DX30',
       'AGE', 'DIED', 'FEMALE', 'LOS', 'PAY1', 'TOTCHG', 'ZIPINC_QRTL',
       'row_id', 'CCI', 'RACE'],
      dtype='object')

In [89]:
useful_cols = demo_cols + ["CCI"]

In [90]:
data = data[useful_cols]

In [91]:
filtered_path = "../../../data/hcup/nis/NIS_2015/total_withage_2015_withCCI_filtered.pickle"

In [92]:
data.to_pickle(filtered_path)

#### Merge data with age

In [13]:
data07_withage = pd.read_pickle('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2007/total_withage_2007_withCCI_filtered.pickle')
data08_withage = pd.read_pickle('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2008/total_withage_2008_withCCI_filtered.pickle')
data09_withage = pd.read_pickle('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2009/total_withage_2009_withCCI_filtered.pickle')
data10_withage = pd.read_pickle('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2010/total_withage_2010_withCCI_filtered.pickle')
data11_withage = pd.read_pickle('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2011/total_withage_2011_withCCI_filtered.pickle')
data12_withage = pd.read_pickle('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2012/total_withage_2012_withCCI_filtered.pickle')
data13_withage = pd.read_pickle('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2013/total_withage_2013_withCCI_filtered.pickle')
data14_withage = pd.read_pickle('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/NIS2014/total_withage_2014_withCCI_filtered.pickle')
data15_withage = pd.read_pickle('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/NIS_2015/total_withage_2015_withCCI_filtered.pickle')

In [14]:
data_withage = data07_withage.append(data08_withage).append(data09_withage).append(data10_withage).append(data11_withage).append(data12_withage).append(data13_withage).append(data14_withage).append(data15_withage)

In [15]:
len(data07_withage) + len(data08_withage) + len(data09_withage) + len(data10_withage) + len(data11_withage) + len(data12_withage) + len(data13_withage) + len(data14_withage) + len(data15_withage)

66647201

In [16]:
len(data_withage)

66647201

In [95]:
path_total = "../../../data/hcup/nis/all_year_combination/total_withage_withCCI_filtered.pickle"

In [96]:
data_withage.to_pickle(path_total)

#### Merge fn_pt with missing died

In [79]:
data07_missingdied = pd.read_csv('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2007/fn_adult_missingdied_2007.csv')
data08_missingdied = pd.read_csv('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2008/fn_adult_missingdied_2008.csv')
data09_missingdied = pd.read_csv('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2009/fn_adult_missingdied_2009.csv')
data10_missingdied = pd.read_csv('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2010/fn_adult_missingdied_2010.csv')
data11_missingdied = pd.read_csv('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2011/fn_adult_missingdied_2011.csv')
data12_missingdied = pd.read_csv('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2012/fn_adult_missingdied_2012.csv')
data13_missingdied = pd.read_csv('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/2013/fn_adult_missingdied_2013.csv')
data14_missingdied = pd.read_csv('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/NIS2014/fn_adult_missingdied_2014.csv')
data15_missingdied = pd.read_csv('/ufrc/djlemas/xinsongdu/jupyter_notebook/data/hcup/nis/NIS_2015/fn_adult_missingdied_2015.csv')

In [80]:
data_missingdied = data07_missingdied.append(data08_missingdied).append(data09_missingdied).append(data10_missingdied).append(data11_missingdied).append(data12_missingdied).append(data13_missingdied).append(data14_missingdied).append(data15_missingdied)

/apps/python3/3.6.5/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [81]:
data_missingdied

,AGE,AGEDAY,AGE_NEONATE,AHAID,AMONTH,APRDRG,APRDRG_Risk_Mortality,APRDRG_Severity,APSDRG,APSDRG_Charge_Weight,APSDRG_LOS_Weight,APSDRG_Mortality_Weight,ASOURCE,ASOURCEUB92,ASOURCE_X,ATYPE,AWEEKEND,BODYSYSTEM1,BODYSYSTEM10,BODYSYSTEM11,BODYSYSTEM12,BODYSYSTEM13,BODYSYSTEM14,BODYSYSTEM15,BODYSYSTEM16,BODYSYSTEM17,BODYSYSTEM18,BODYSYSTEM19,BODYSYSTEM2,BODYSYSTEM20,BODYSYSTEM21,BODYSYSTEM22,BODYSYSTEM23,BODYSYSTEM24,BODYSYSTEM25,BODYSYSTEM26,BODYSYSTEM27,BODYSYSTEM28,BODYSYSTEM29,BODYSYSTEM3,BODYSYSTEM30,BODYSYSTEM4,BODYSYSTEM5,BODYSYSTEM6,BODYSYSTEM7,BODYSYSTEM8,BODYSYSTEM9,CHRON1,CHRON10,CHRON11,CHRON12,CHRON13,CHRON14,CHRON15,CHRON16,CHRON17,CHRON18,CHRON19,CHRON2,CHRON20,CHRON21,CHRON22,CHRON23,CHRON24,CHRON25,CHRON26,CHRON27,CHRON28,CHRON29,CHRON3,CHRON30,CHRON4,CHRON5,CHRON6,CHRON7,CHRON8,CHRON9,CHRONB1,CHRONB10,CHRONB11,CHRONB12,CHRONB13,CHRONB14,CHRONB15,CHRONB16,CHRONB17,CHRONB18,CHRONB19,CHRONB2,CHRONB20,CHRONB21,CHRONB22,CHRONB23,CHRONB24,CHRONB25,CHRONB3,CHRONB4,CHRONB5,CHRONB6,CHRONB7,CHRONB8,CHRONB9,CM_AIDS,CM_ALCOHOL,CM_ANEMDEF,CM_ARTH,CM_BLDLOSS,CM_CHF,CM_CHRNLUNG,CM_COAG,CM_DEPRESS,CM_DM,CM_DMCX,CM_DRUG,CM_HTN_C,CM_HYPOTHY,CM_LIVER,CM_LYMPH,CM_LYTES,CM_METS,CM_NEURO,CM_OBESE,CM_PARA,CM_PERIVASC,CM_PSYCH,CM_PULMCIRC,CM_RENLFAIL,CM_TUMOR,CM_ULCER,CM_VALVE,CM_WGHTLOSS,DIED,DISCWT,DISCWT_x,DISCWT_y,DISPUB04,DISPUNIFORM,DQTR,DQTR_X,DRG,DRG24,DRGVER,DRG_NoPOA,DSHOSPID,DS_DX_Category1,DS_LOS_Level,DS_LOS_Scale,DS_Mrt_Level,DS_Mrt_Scale,DS_RD_Level,DS_RD_Scale,DS_Stage1,DX1,DX10,DX11,DX12,DX13,DX14,DX15,DX16,DX17,DX18,DX19,DX2,DX20,DX21,DX22,DX23,DX24,DX25,DX26,DX27,DX28,DX29,DX3,DX30,DX4,DX5,DX6,DX7,DX8,DX9,DXCCS1,DXCCS10,DXCCS11,DXCCS12,DXCCS13,DXCCS14,DXCCS15,DXCCS16,DXCCS17,DXCCS18,DXCCS19,DXCCS2,DXCCS20,DXCCS21,DXCCS22,DXCCS23,DXCCS24,DXCCS25,DXCCS26,DXCCS27,DXCCS28,DXCCS29,DXCCS3,DXCCS30,DXCCS4,DXCCS5,DXCCS6,DXCCS7,DXCCS8,DXCCS9,DXMCCS1,DXVER,ECODE1,ECODE2,ECODE3,ECODE4,ELECTIVE,E_CCS1,E_CCS2,E_CCS3,E_CCS4,E_MCCS1,FEMALE,HCUP_ED,HFIPSSTCO,HOSPADDR,HOSPBRTH,HOSPCITY,HOSPID,HOSPID_x,HOSPID_y,HOSPNAME,HOSPST,HOSPSTCO,HOSPST_x,HOSPST_y,HOSPWT,HOSPZIP,HOSP_BEDSIZE,HOSP_CONTROL,HOSP_DIVISION,HOSP_LOCATION,HOSP_LOCTEACH,HOSP_LPNFTEAPD,HOSP_MHSCLUSTER,HOSP_MHSMEMBER,HOSP_NAFTEAPD,HOSP_NIS,HOSP_OPSURGPCT,HOSP_REGION,HOSP_RNFTEAPD,HOSP_RNPCT,HOSP_TEACH,H_CONTRL,IDNUMBER,KEY,KEY_NIS,LOS,LOS_X,MDC,MDC24,MDC_NoPOA,MDNUM1_R,MDNUM2_R,NCHRONIC,NDX,NECODE,NEOMAT,NIS_STRATUM,NIS_STRATUM_x,NIS_STRATUM_y,NPR,N_DISC_U,N_HOSP_U,ORPROC,PAY1,PAY1_X,PAY2,PAY2_X,PCLASS1,PCLASS10,PCLASS11,PCLASS12,PCLASS13,PCLASS14,PCLASS15,PCLASS2,PCLASS3,PCLASS4,PCLASS5,PCLASS6,PCLASS7,PCLASS8,PCLASS9,PL_NCHS,PL_NCHS2006,PR1,PR10,PR11,PR12,PR13,PR14,PR15,PR2,PR3,PR4,PR5,PR6,PR7,PR8,PR9,PRCCS1,PRCCS10,PRCCS11,PRCCS12,PRCCS13,PRCCS14,PRCCS15,PRCCS2,PRCCS3,PRCCS4,PRCCS5,PRCCS6,PRCCS7,PRCCS8,PRCCS9,PRDAY1,PRDAY10,PRDAY11,PRDAY12,PRDAY13,PRDAY14,PRDAY15,PRDAY2,PRDAY3,PRDAY4,PRDAY5,PRDAY6,PRDAY7,PRDAY8,PRDAY9,PRMCCS1,PRVER,PointOfOriginUB04,PointOfOrigin_X,RACE,SERVICELINE,S_DISC_U,S_HOSP_U,TOTAL_DISC,TOTCHG,TOTCHG_X,TRAN_IN,TRAN_OUT,YEAR,YEAR_x,YEAR_y,ZIPINC_QRTL
0,75.0,-99.0,NaN,6140016.0,3,660,3,4,5741.0,0.99212,1.08538,0.67070,5.0,2.0,2,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,0,-99,-99,-99,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,1,0,0,1,1,4,3,4,16,-99,-99,-99,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,17,7,9,2,16,2,7,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-9,6.033394,NaN,NaN,-9.0,-9,1,1.0,574,574,24,NaN,145.0,HEM02,5.0,279.83719,5.0,1336.50671,5.0,361.61456,3.03,2848,2749,28800,7806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99591,NaN,4280,56212,23875,78650,185,43811,59,54,63,246,-99,-99,-99,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244,NaN,108,146,44,102,29,113,NaN,NaN,NaN,NaN,NaN,NaN,0,-999,-999,-999,-999,NaN,0,1,25005.0,363 Highland Avenue,0,Fall River,25187.0,NaN,NaN,Southcoast

In [82]:
data_missingdied.to_pickle("../../../data/hcup/nis/all_year_combination/fn_pt_missingdied.pickle")